In [149]:
import csv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import keras
from keras.layers import Input, Dense, LSTM, Dropout
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from keras.callbacks import EarlyStopping, ModelCheckpoint
from random import randint
import argparse

In [150]:
data = pd.read_excel('final_version.xlsx', header=None)
data = data.to_numpy()[1:]

In [151]:
data

array([[1960, 1, 1, -1.6, 0],
       [1960, 1, 2, -1.9, 0.4],
       [1960, 1, 3, 4, 0],
       ...,
       [2019, 12, 29, 3.8, 1.4],
       [2019, 12, 30, 2.7, 0.4],
       [2019, 12, 31, -7.9, 0]], dtype=object)

In [152]:
idx = 0

In [153]:
X, Y = [], []

while True:
    if idx+15 >= len(data):
        break
    X.append(data[idx:idx+15])
    Y.append(data[idx+15])
    idx += 1

In [154]:
X = np.array(X)
Y = np.array(Y)

In [155]:
np.shape(X)

(21900, 15, 5)

In [156]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [128]:
np.shape(X_train)

(25, 15, 5)

In [157]:
X_train = np.array(X_train)[:, :, [1, 3, 4]]
Y_train = np.array(Y_train)[:, -1].reshape(-1, 1)
X_test = np.array(X_test)[:, :, [1, 3, 4]]
Y_test = np.array(Y_test)[:, -1].reshape(-1, 1)

In [160]:
X_nonzero = []
Y_nonzero = []
for i, y in enumerate(Y_train):
    if y[-1] != 0.0:
        X_nonzero.append(X_train[i])
        Y_nonzero.append(y)

In [161]:
X_train = np.concatenate((X_train, X_nonzero, X_nonzero, X_nonzero))
Y_train = np.concatenate((Y_train, Y_nonzero, Y_nonzero, Y_nonzero))

In [162]:
def build_model(input_shape):
    x = Input(shape=input_shape)

    # h = LSTM(20, return_sequences=True, activation='tanh', kernel_initializer='he_normal')(x)
    # h = LSTM(20, return_sequences=True, activation='tanh', kernel_initializer='he_normal')(h)
    # h = LSTM(20, activation='tanh', kernel_initializer='he_normal')(h)
    h = LSTM(160, activation='tanh',
             kernel_initializer='he_normal')(x)  # 128

    y = Dense(1, activation='relu')(h)
    return Model(inputs=x, outputs=y)

In [163]:
model = build_model((15, 3))
model.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 15, 3)             0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 160)               104960    
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 161       
Total params: 105,121
Trainable params: 105,121
Non-trainable params: 0
_________________________________________________________________


In [164]:
model.compile(loss='mse', optimizer=RMSprop(), metrics=['mae'])


In [166]:
es = EarlyStopping(monitor='val_mae', mode='min', verbose=1, patience=1000)
mc = ModelCheckpoint('best_model.h5', monitor='val_mae', mode='min', save_best_only=True)

In [167]:
hist = model.fit(
    X_train, Y_train,
    batch_size=128,
    validation_split=0.2,
    epochs=1000,
    verbose=2,
    callbacks=[es, mc])

Train on 26584 samples, validate on 6647 samples
Epoch 1/1000
 - 9s - loss: 368.0700 - mae: 7.8354 - val_loss: 571.4264 - val_mae: 11.2164
Epoch 2/1000
 - 8s - loss: 338.8809 - mae: 8.2807 - val_loss: 541.3898 - val_mae: 11.3413
Epoch 3/1000
 - 9s - loss: 329.6630 - mae: 8.3408 - val_loss: 536.1862 - val_mae: 11.2931
Epoch 4/1000
 - 8s - loss: 324.8028 - mae: 8.3013 - val_loss: 526.7047 - val_mae: 11.3348
Epoch 5/1000
 - 8s - loss: 321.4934 - mae: 8.2717 - val_loss: 515.5028 - val_mae: 11.3854
Epoch 6/1000
 - 8s - loss: 319.0707 - mae: 8.2575 - val_loss: 514.9921 - val_mae: 11.3447
Epoch 7/1000
 - 8s - loss: 317.0978 - mae: 8.1903 - val_loss: 510.0927 - val_mae: 11.3060
Epoch 8/1000
 - 8s - loss: 314.9778 - mae: 8.1589 - val_loss: 509.5600 - val_mae: 11.2147
Epoch 9/1000
 - 9s - loss: 313.0414 - mae: 8.1171 - val_loss: 512.4099 - val_mae: 11.1613
Epoch 10/1000
 - 8s - loss: 311.3492 - mae: 8.0488 - val_loss: 503.2046 - val_mae: 11.1098
Epoch 11/1000
 - 8s - loss: 309.2136 - mae: 7.9878

Epoch 92/1000
 - 9s - loss: 44.1495 - mae: 2.7346 - val_loss: 70.6441 - val_mae: 4.3049
Epoch 93/1000
 - 9s - loss: 44.4679 - mae: 2.7326 - val_loss: 76.3126 - val_mae: 4.6299
Epoch 94/1000
 - 10s - loss: 41.9371 - mae: 2.6840 - val_loss: 66.8989 - val_mae: 4.2798
Epoch 95/1000
 - 10s - loss: 41.4250 - mae: 2.6629 - val_loss: 81.7909 - val_mae: 4.7640
Epoch 96/1000
 - 9s - loss: 41.7224 - mae: 2.6751 - val_loss: 64.0786 - val_mae: 4.2250
Epoch 97/1000
 - 10s - loss: 39.4720 - mae: 2.6132 - val_loss: 66.2261 - val_mae: 4.3053
Epoch 98/1000
 - 9s - loss: 39.4464 - mae: 2.6112 - val_loss: 72.0991 - val_mae: 4.6786
Epoch 99/1000
 - 8s - loss: 38.0325 - mae: 2.5645 - val_loss: 61.5122 - val_mae: 4.2393
Epoch 100/1000
 - 10s - loss: 37.1184 - mae: 2.5390 - val_loss: 59.4783 - val_mae: 4.0767
Epoch 101/1000
 - 9s - loss: 36.6866 - mae: 2.5114 - val_loss: 64.3106 - val_mae: 4.2656
Epoch 102/1000
 - 8s - loss: 35.6713 - mae: 2.4880 - val_loss: 59.4890 - val_mae: 4.1242
Epoch 103/1000
 - 8s - lo

 - 8s - loss: 14.3957 - mae: 1.5850 - val_loss: 29.0847 - val_mae: 2.9172
Epoch 185/1000
 - 9s - loss: 14.5357 - mae: 1.5956 - val_loss: 28.0950 - val_mae: 2.8494
Epoch 186/1000
 - 10s - loss: 14.2746 - mae: 1.5789 - val_loss: 28.8539 - val_mae: 2.9510
Epoch 187/1000
 - 9s - loss: 14.3285 - mae: 1.5764 - val_loss: 26.1521 - val_mae: 2.7739
Epoch 188/1000
 - 9s - loss: 14.2759 - mae: 1.5777 - val_loss: 25.5487 - val_mae: 2.7231
Epoch 189/1000
 - 9s - loss: 14.0669 - mae: 1.5641 - val_loss: 30.0059 - val_mae: 3.1046
Epoch 190/1000
 - 8s - loss: 13.9756 - mae: 1.5809 - val_loss: 28.9605 - val_mae: 2.9829
Epoch 191/1000
 - 9s - loss: 13.9910 - mae: 1.5620 - val_loss: 28.2568 - val_mae: 2.8844
Epoch 192/1000
 - 9s - loss: 14.0378 - mae: 1.5619 - val_loss: 25.6114 - val_mae: 2.7440
Epoch 193/1000
 - 9s - loss: 13.9075 - mae: 1.5455 - val_loss: 27.2472 - val_mae: 2.8679
Epoch 194/1000
 - 8s - loss: 13.7690 - mae: 1.5506 - val_loss: 26.0774 - val_mae: 2.7958
Epoch 195/1000
 - 9s - loss: 13.710

Epoch 277/1000
 - 6s - loss: 10.5695 - mae: 1.2864 - val_loss: 21.5265 - val_mae: 2.4148
Epoch 278/1000
 - 6s - loss: 10.6611 - mae: 1.2871 - val_loss: 24.6278 - val_mae: 2.6801
Epoch 279/1000
 - 6s - loss: 10.6613 - mae: 1.2847 - val_loss: 20.9828 - val_mae: 2.3552
Epoch 280/1000
 - 5s - loss: 10.5066 - mae: 1.2737 - val_loss: 21.0256 - val_mae: 2.3469
Epoch 281/1000
 - 5s - loss: 10.6647 - mae: 1.2814 - val_loss: 20.6584 - val_mae: 2.3291
Epoch 282/1000
 - 5s - loss: 10.5311 - mae: 1.2736 - val_loss: 24.9340 - val_mae: 2.6350
Epoch 283/1000
 - 5s - loss: 10.5871 - mae: 1.2812 - val_loss: 21.6488 - val_mae: 2.4225
Epoch 284/1000
 - 5s - loss: 10.5576 - mae: 1.2788 - val_loss: 22.3016 - val_mae: 2.4512
Epoch 285/1000
 - 5s - loss: 10.4758 - mae: 1.2714 - val_loss: 21.5331 - val_mae: 2.4212
Epoch 286/1000
 - 6s - loss: 10.4071 - mae: 1.2633 - val_loss: 21.3945 - val_mae: 2.4040
Epoch 287/1000
 - 7s - loss: 10.4506 - mae: 1.2643 - val_loss: 26.7379 - val_mae: 2.8095
Epoch 288/1000
 - 6s 

Epoch 370/1000
 - 8s - loss: 9.4311 - mae: 1.1554 - val_loss: 19.8879 - val_mae: 2.2333
Epoch 371/1000
 - 8s - loss: 9.3366 - mae: 1.1407 - val_loss: 18.9036 - val_mae: 2.1403
Epoch 372/1000
 - 9s - loss: 9.3813 - mae: 1.1466 - val_loss: 18.5901 - val_mae: 2.0778
Epoch 373/1000
 - 9s - loss: 9.3826 - mae: 1.1396 - val_loss: 19.5853 - val_mae: 2.2086
Epoch 374/1000
 - 8s - loss: 9.3910 - mae: 1.1486 - val_loss: 19.1587 - val_mae: 2.1720
Epoch 375/1000
 - 9s - loss: 9.4431 - mae: 1.1510 - val_loss: 19.9140 - val_mae: 2.2475
Epoch 376/1000
 - 8s - loss: 9.3458 - mae: 1.1381 - val_loss: 18.7557 - val_mae: 2.1139
Epoch 377/1000
 - 9s - loss: 9.2952 - mae: 1.1371 - val_loss: 18.8422 - val_mae: 2.1152
Epoch 378/1000
 - 9s - loss: 9.2814 - mae: 1.1338 - val_loss: 19.2080 - val_mae: 2.1524
Epoch 379/1000
 - 10s - loss: 9.2557 - mae: 1.1359 - val_loss: 19.0810 - val_mae: 2.1448
Epoch 380/1000
 - 9s - loss: 9.2588 - mae: 1.1280 - val_loss: 20.8095 - val_mae: 2.3587
Epoch 381/1000
 - 8s - loss: 9.

...

Epoch 557/1000
 - 8s - loss: 8.2996 - mae: 0.9914 - val_loss: 17.3465 - val_mae: 1.8969
Epoch 558/1000
 - 9s - loss: 8.2962 - mae: 0.9895 - val_loss: 17.2038 - val_mae: 1.8688
Epoch 559/1000
 - 8s - loss: 8.2816 - mae: 0.9948 - val_loss: 17.4668 - val_mae: 1.9182
Epoch 560/1000
 - 8s - loss: 8.3200 - mae: 0.9907 - val_loss: 17.2618 - val_mae: 1.8844
Epoch 561/1000
 - 8s - loss: 8.2616 - mae: 0.9905 - val_loss: 17.0348 - val_mae: 1.8428
Epoch 562/1000
 - 8s - loss: 8.2639 - mae: 0.9853 - val_loss: 17.5125 - val_mae: 1.9434
Epoch 563/1000
 - 8s - loss: 8.2338 - mae: 0.9869 - val_loss: 17.2992 - val_mae: 1.8931
Epoch 564/1000
 - 9s - loss: 8.2611 - mae: 0.9862 - val_loss: 17.4683 - val_mae: 1.9120
Epoch 565/1000
 - 9s - loss: 8.2350 - mae: 0.9842 - val_loss: 17.5294 - val_mae: 1.9319
Epoch 566/1000
 - 9s - loss: 8.2872 - mae: 0.9947 - val_loss: 17.5644 - val_mae: 1.9286
Epoch 567/1000
 - 9s - loss: 8.2700 - mae: 0.9853 - val_loss: 17.1297 - val_mae: 1.8640
Epoch 568/1000
 - 9s - loss: 8.3

 - 6s - loss: 8.0536 - mae: 0.9431 - val_loss: 17.3504 - val_mae: 1.9005
Epoch 651/1000
 - 9s - loss: 8.0430 - mae: 0.9461 - val_loss: 17.0168 - val_mae: 1.8371
Epoch 652/1000
 - 8s - loss: 8.0562 - mae: 0.9461 - val_loss: 17.0946 - val_mae: 1.8536
Epoch 653/1000
 - 9s - loss: 8.0545 - mae: 0.9481 - val_loss: 16.7381 - val_mae: 1.7711
Epoch 654/1000
 - 9s - loss: 8.0582 - mae: 0.9453 - val_loss: 18.8694 - val_mae: 2.0809
Epoch 655/1000
 - 8s - loss: 7.9805 - mae: 0.9347 - val_loss: 17.4073 - val_mae: 1.9027
Epoch 656/1000
 - 8s - loss: 8.0216 - mae: 0.9415 - val_loss: 17.5630 - val_mae: 1.9313
Epoch 657/1000
 - 8s - loss: 8.0704 - mae: 0.9479 - val_loss: 18.0922 - val_mae: 1.9018
Epoch 658/1000
 - 8s - loss: 8.0301 - mae: 0.9442 - val_loss: 16.9533 - val_mae: 1.8254
Epoch 659/1000
 - 8s - loss: 8.0475 - mae: 0.9465 - val_loss: 18.2432 - val_mae: 1.9427
Epoch 660/1000
 - 8s - loss: 8.0661 - mae: 0.9473 - val_loss: 17.0448 - val_mae: 1.8528
Epoch 661/1000
 - 8s - loss: 8.0591 - mae: 0.94

Epoch 744/1000
 - 8s - loss: 7.7542 - mae: 0.9098 - val_loss: 16.0164 - val_mae: 1.7119
Epoch 745/1000
 - 8s - loss: 7.7894 - mae: 0.9162 - val_loss: 16.5920 - val_mae: 1.8446
Epoch 746/1000
 - 8s - loss: 7.7935 - mae: 0.9068 - val_loss: 16.1822 - val_mae: 1.7584
Epoch 747/1000
 - 8s - loss: 7.7857 - mae: 0.9108 - val_loss: 16.6373 - val_mae: 1.8569
Epoch 748/1000
 - 8s - loss: 7.7634 - mae: 0.9109 - val_loss: 16.1389 - val_mae: 1.7325
Epoch 749/1000
 - 8s - loss: 7.8335 - mae: 0.9145 - val_loss: 16.2394 - val_mae: 1.7519
Epoch 750/1000
 - 8s - loss: 7.7661 - mae: 0.9114 - val_loss: 16.1292 - val_mae: 1.7430
Epoch 751/1000
 - 8s - loss: 7.7664 - mae: 0.9140 - val_loss: 16.4679 - val_mae: 1.7979
Epoch 752/1000
 - 8s - loss: 7.7628 - mae: 0.9065 - val_loss: 16.4374 - val_mae: 1.8181
Epoch 753/1000
 - 8s - loss: 7.7630 - mae: 0.9068 - val_loss: 16.1456 - val_mae: 1.7406
Epoch 754/1000
 - 8s - loss: 7.7347 - mae: 0.9041 - val_loss: 16.3101 - val_mae: 1.7985
Epoch 755/1000
 - 8s - loss: 7.7

 - 8s - loss: 7.2959 - mae: 0.8662 - val_loss: 15.7395 - val_mae: 1.7486
Epoch 838/1000
 - 8s - loss: 7.3177 - mae: 0.8677 - val_loss: 15.4990 - val_mae: 1.6794
Epoch 839/1000
 - 8s - loss: 7.3193 - mae: 0.8616 - val_loss: 15.8057 - val_mae: 1.7513
Epoch 840/1000
 - 8s - loss: 7.3150 - mae: 0.8680 - val_loss: 15.9036 - val_mae: 1.7532
Epoch 841/1000
 - 8s - loss: 7.2941 - mae: 0.8655 - val_loss: 15.6212 - val_mae: 1.7225
Epoch 842/1000
 - 8s - loss: 7.3221 - mae: 0.8693 - val_loss: 15.4435 - val_mae: 1.6865
Epoch 843/1000
 - 8s - loss: 7.3003 - mae: 0.8624 - val_loss: 15.7138 - val_mae: 1.7254
Epoch 844/1000
 - 8s - loss: 7.3286 - mae: 0.8689 - val_loss: 15.5740 - val_mae: 1.7114
Epoch 845/1000
 - 8s - loss: 7.3196 - mae: 0.8690 - val_loss: 15.8979 - val_mae: 1.7958
Epoch 846/1000
 - 8s - loss: 7.2883 - mae: 0.8628 - val_loss: 15.6061 - val_mae: 1.7336
Epoch 847/1000
 - 8s - loss: 7.3193 - mae: 0.8643 - val_loss: 15.5727 - val_mae: 1.7065
Epoch 848/1000
 - 8s - loss: 7.3344 - mae: 0.86

Epoch 931/1000
 - 8s - loss: 7.0125 - mae: 0.8389 - val_loss: 14.4858 - val_mae: 1.6135
Epoch 932/1000
 - 8s - loss: 7.0086 - mae: 0.8376 - val_loss: 14.6082 - val_mae: 1.6531
Epoch 933/1000
 - 8s - loss: 7.0169 - mae: 0.8409 - val_loss: 14.4165 - val_mae: 1.6034
Epoch 934/1000
 - 8s - loss: 7.0419 - mae: 0.8396 - val_loss: 14.4222 - val_mae: 1.6008
Epoch 935/1000
 - 8s - loss: 6.9944 - mae: 0.8362 - val_loss: 14.4919 - val_mae: 1.6258
Epoch 936/1000
 - 8s - loss: 7.0457 - mae: 0.8383 - val_loss: 14.6485 - val_mae: 1.6600
Epoch 937/1000
 - 8s - loss: 7.0817 - mae: 0.8385 - val_loss: 14.6066 - val_mae: 1.6455
Epoch 938/1000
 - 8s - loss: 7.0247 - mae: 0.8428 - val_loss: 14.6923 - val_mae: 1.6764
Epoch 939/1000
 - 8s - loss: 7.0238 - mae: 0.8402 - val_loss: 14.5216 - val_mae: 1.6337
Epoch 940/1000
 - 8s - loss: 7.0418 - mae: 0.8428 - val_loss: 14.4237 - val_mae: 1.5973
Epoch 941/1000
 - 8s - loss: 7.0104 - mae: 0.8358 - val_loss: 14.8075 - val_mae: 1.6950
Epoch 942/1000
 - 8s - loss: 7.0

In [133]:
model.load_weights('./best_model_20000_80_RMSprop'+'.h5')


In [134]:
loss, mae = model.evaluate(X_test, Y_test)
print(loss, mae)


7/7 [==============================] - 0s 11ms/step
0.08496123552322388 0.19529342651367188


In [144]:
samples = [randint(0, len(X_test)-1) for _ in range(150)]
pred = model.predict([X_test[samples]])
pred = pred.reshape(-1)
real = Y_test[samples]
real = real.reshape(-1)

res = np.abs(pred - real)
print(np.mean(res), np.max(res), np.min(res))


0.18894030888875354 0.6 0.0


In [145]:
pred


array([0.       , 0.       , 2.067054 , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 2.067054 , 0.       , 0.       ,
       0.       , 0.       , 2.067054 , 0.       , 0.       , 0.       ,
       0.       , 2.067054 , 0.       , 2.067054 , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 2.067054 ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 2.067054 , 0.       , 0.       ,
       2.067054 , 0.       , 2.0670426, 2.0670426, 0.       , 0.       ,
       2.067054 , 2.067054 , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 2.067054 , 0.       , 0.       ,
       2.067054 , 0.       , 0.       , 0.       , 

In [146]:
real


array([0.1, 0.6, 1.7, 0.6, 0.6, 0.1, 0.6, 0, 0.6, 0.1, 0, 0, 0, 0, 0.1,
       1.7, 0.3, 0, 0.6, 0.1, 1.7, 0, 0.6, 0, 0.1, 1.7, 0, 1.7, 0, 0.6,
       0.1, 0, 0, 0, 0.6, 1.7, 0.6, 0, 0.3, 0, 0, 0, 0.6, 0, 0, 0.1, 0.1,
       0, 0, 0, 0, 0, 0.6, 0.3, 0.6, 0, 0, 1.7, 0, 0, 1.7, 0, 1.7, 1.7, 0,
       0.6, 1.7, 1.7, 0, 0.3, 0.1, 0.3, 0.1, 0, 0.6, 1.7, 0, 0, 1.7, 0.3,
       0.6, 0.6, 0.3, 0, 0, 0, 0, 1.7, 0, 0, 0, 0.1, 0, 1.7, 0, 0.1, 0,
       0.3, 0.3, 0, 1.7, 0, 0.3, 0.1, 0, 0.6, 0.3, 0, 0.3, 0, 0, 0.3, 0,
       0.1, 0.3, 0.3, 0, 0.1, 0, 0.6, 0, 0, 0.6, 0, 1.7, 0, 1.7, 0.1, 0.1,
       0.3, 0, 0.3, 0, 0, 0.1, 0.1, 0.6, 0, 0, 0.1, 0, 0.3, 0, 0, 0, 0,
       0.3, 0, 0.6, 1.7], dtype=object)

In [19]:
X2 = X[:, :, [1, 3, 4]]

In [20]:
np.shape(X2)
np.shape([X2[-1,:,:]])


(1, 15, 3)

In [21]:
X3=[X2[-1,:,:]]
X3


[array([[12, 2, 0],
        [12, 2, 0.3],
        [12, -1.7, 0],
        [12, 1.1, 0],
        [12, 3.8, 1.4],
        [12, 2.7, 0.4],
        [12, -7.9, 0],
        [1, -2.2, 0],
        [1, 1, 0],
        [1, -0.1, 0],
        [1, 1.2, 0],
        [1, 1.3, 0],
        [1, 3.1, 0],
        [1, 6.3, 46.3],
        [1, 2.4, 0]], dtype=object)]

In [22]:
np.shape(X3)


(1, 15, 3)

In [23]:
X3=X2[38:-1,:,:]
pred = model.predict([X3])
pred = pred.reshape(-1)
pred
X3


array([[[12, 3.4, 2.9],
        [12, 0.8, 0],
        [12, 2, 0],
        [12, 2, 0.3],
        [12, -1.7, 0],
        [12, 1.1, 0],
        [12, 3.8, 1.4],
        [12, 2.7, 0.4],
        [12, -7.9, 0],
        [1, -2.2, 0],
        [1, 1, 0],
        [1, -0.1, 0],
        [1, 1.2, 0],
        [1, 1.3, 0],
        [1, 3.1, 0]],

       [[12, 0.8, 0],
        [12, 2, 0],
        [12, 2, 0.3],
        [12, -1.7, 0],
        [12, 1.1, 0],
        [12, 3.8, 1.4],
        [12, 2.7, 0.4],
        [12, -7.9, 0],
        [1, -2.2, 0],
        [1, 1, 0],
        [1, -0.1, 0],
        [1, 1.2, 0],
        [1, 1.3, 0],
        [1, 3.1, 0],
        [1, 6.3, 46.3]]], dtype=object)

In [24]:
pred


array([0., 0.], dtype=float32)

In [25]:
X4=[X2[-1,:,:]]
pred=model.predict([X4])
pred=pred.reshape(-1)
pred

array([0.], dtype=float32)

In [26]:
X4

[array([[12, 2, 0],
        [12, 2, 0.3],
        [12, -1.7, 0],
        [12, 1.1, 0],
        [12, 3.8, 1.4],
        [12, 2.7, 0.4],
        [12, -7.9, 0],
        [1, -2.2, 0],
        [1, 1, 0],
        [1, -0.1, 0],
        [1, 1.2, 0],
        [1, 1.3, 0],
        [1, 3.1, 0],
        [1, 6.3, 46.3],
        [1, 2.4, 0]], dtype=object)]

In [27]:
np.shape(X2)

(41, 15, 3)

In [28]:
X5=[X2[29,:,:]]

In [29]:
X5

[array([[12, 2.7, 0],
        [12, 5.3, 0],
        [12, 7.8, 0],
        [12, 7.7, 9.8],
        [12, 0.8, 0],
        [12, -0.8, 0],
        [12, -1.1, 0],
        [12, -1.2, 0.6],
        [12, 4.1, 0.3],
        [12, 3.4, 2.9],
        [12, 0.8, 0],
        [12, 2, 0],
        [12, 2, 0.3],
        [12, -1.7, 0],
        [12, 1.1, 0]], dtype=object)]

In [30]:
X5=[X2[29,:,:]]
pred=model.predict([X5])
pred=pred.reshape(-1)
pred


array([0.], dtype=float32)

In [31]:
[X2[:,:,:]]
np.shape(X2[:,:,:])


(41, 15, 3)

In [32]:
X6=X2[:,:,:]
pred=model.predict([X6])
pred=pred.reshape(-1)
pred

array([ 0.      ,  0.      ,  0.      , 10.300491,  0.      ,  0.      ,
        0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,
        0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,
        0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,
        0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,
        0.      ,  8.072527,  0.      ,  0.      ,  0.      ,  0.      ,
        0.      ,  0.      ,  0.      ,  0.      ,  0.      ],
      dtype=float32)

In [33]:
X6

array([[[11, 5.6, 18.8],
        [11, 6.8, 0],
        [11, 8.3, 21.1],
        ...,
        [11, 3.1, 0],
        [11, 3.5, 0],
        [11, 1.5, 0]],

       [[11, 6.8, 0],
        [11, 8.3, 21.1],
        [11, 4.6, 14.7],
        ...,
        [11, 3.5, 0],
        [11, 1.5, 0],
        [11, 4.1, 0]],

       [[11, 8.3, 21.1],
        [11, 4.6, 14.7],
        [11, 0.2, 0],
        ...,
        [11, 1.5, 0],
        [11, 4.1, 0],
        [12, 4.6, 3.3]],

       ...,

       [[12, 3.4, 2.9],
        [12, 0.8, 0],
        [12, 2, 0],
        ...,
        [1, 1.2, 0],
        [1, 1.3, 0],
        [1, 3.1, 0]],

       [[12, 0.8, 0],
        [12, 2, 0],
        [12, 2, 0.3],
        ...,
        [1, 1.3, 0],
        [1, 3.1, 0],
        [1, 6.3, 46.3]],

       [[12, 2, 0],
        [12, 2, 0.3],
        [12, -1.7, 0],
        ...,
        [1, 3.1, 0],
        [1, 6.3, 46.3],
        [1, 2.4, 0]]], dtype=object)

In [35]:
print(pred)

[ 0.        0.        0.       10.300491  0.        0.        0.
  0.        0.        0.        0.        0.        0.        0.
  0.        0.        0.        0.        0.        0.        0.
  0.        0.        0.        0.        0.        0.        0.
  0.        0.        0.        8.072527  0.        0.        0.
  0.        0.        0.        0.        0.        0.      ]


In [37]:
np.shape(Y_test)

(9, 1)

In [147]:
def build_model(input_shape):
    x = Input(shape=input_shape)

    # h = LSTM(20, return_sequences=True, activation='tanh', kernel_initializer='he_normal')(x)
    # h = LSTM(20, return_sequences=True, activation='tanh', kernel_initializer='he_normal')(h)
    # h = LSTM(20, activation='tanh', kernel_initializer='he_normal')(h)
    h = LSTM(160, activation='tanh',
             kernel_initializer='he_normal')(x)  # 128

    y = Dense(1, activation='relu')(h)
    return Model(inputs=x, outputs=y)

In [148]:
model = build_model((15, 3))
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 15, 3)             0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 160)               104960    
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 161       
Total params: 105,121
Trainable params: 105,121
Non-trainable params: 0
_________________________________________________________________
